In [4]:
import pandas as pd
import numpy as np

In [6]:
res_df = pd.read_csv("C:/Users/Bibek/OneDrive/Desktop/DA Training/Restaurant Sales Data Cleaning/restaurant_sales_data.csv")

In [7]:
res_df.head()

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
0,ORD_705844,CUST_092,Side Dishes,Side Salad,3.0,1.0,3.0,2023-12-21,Credit Card
1,ORD_338528,CUST_021,Side Dishes,Mashed Potatoes,4.0,3.0,12.0,2023-05-19,Digital Wallet
2,ORD_443849,CUST_029,Main Dishes,Grilled Chicken,15.0,4.0,60.0,2023-09-27,Credit Card
3,ORD_630508,CUST_075,Drinks,NaN,NaN,2.0,5.0,2022-08-09,Credit Card
4,ORD_648269,CUST_031,Main Dishes,Pasta Alfredo,12.0,4.0,48.0,2022-05-15,Cash


In [8]:
res_df.isna().sum()

Order ID             0
Customer ID          0
Category             0
Item              1758
Price              876
Quantity           430
Order Total        430
Order Date           0
Payment Method    1082
dtype: int64

In [9]:
res_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17534 entries, 0 to 17533
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Order ID        17534 non-null  object 
 1   Customer ID     17534 non-null  object 
 2   Category        17534 non-null  object 
 3   Item            15776 non-null  object 
 4   Price           16658 non-null  float64
 5   Quantity        17104 non-null  float64
 6   Order Total     17104 non-null  float64
 7   Order Date      17534 non-null  object 
 8   Payment Method  16452 non-null  object 
dtypes: float64(3), object(6)
memory usage: 1.2+ MB


In [10]:
# changing the order date to datetime format

res_df['Order Date'] = pd.to_datetime(res_df['Order Date'])

In [11]:
res_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17534 entries, 0 to 17533
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Order ID        17534 non-null  object        
 1   Customer ID     17534 non-null  object        
 2   Category        17534 non-null  object        
 3   Item            15776 non-null  object        
 4   Price           16658 non-null  float64       
 5   Quantity        17104 non-null  float64       
 6   Order Total     17104 non-null  float64       
 7   Order Date      17534 non-null  datetime64[ns]
 8   Payment Method  16452 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 1.2+ MB


In [12]:
res_df.isna().sum()

Order ID             0
Customer ID          0
Category             0
Item              1758
Price              876
Quantity           430
Order Total        430
Order Date           0
Payment Method    1082
dtype: int64

### Let's see items whose price and quantity are null

In [13]:
res_df[(res_df['Item'].isna()) & (~res_df['Price'].isna()) & (~res_df['Quantity'].isna())]

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
7,ORD_146656,CUST_077,Main Dishes,NaN,15.0,3.0,45.0,2023-02-15,Cash
8,ORD_428611,CUST_083,Desserts,NaN,6.0,2.0,12.0,2023-12-16,Cash
18,ORD_337802,CUST_034,Desserts,NaN,4.0,5.0,20.0,2022-11-13,Credit Card
35,ORD_193762,CUST_028,Main Dishes,NaN,18.0,5.0,90.0,2022-04-08,NaN
51,ORD_782727,CUST_066,Starters,NaN,4.0,4.0,16.0,2022-08-11,Digital Wallet
...,...,...,...,...,...,...,...,...,...
17445,ORD_579430,CUST_016,Drinks,NaN,1.0,3.0,3.0,2022-07-25,NaN
17492,ORD_360934,CUST_080,Drinks,NaN,1.0,3.0,3.0,2022-12-21,Digital Wallet
17505,ORD_605079,CUST_089,Main Dishes,NaN,15.0,4.0,60.0,2022-12-23,Cash
17515,ORD_773375,CUST_024,Drinks,NaN,2.5,5.0,12.5,2022-11-24,NaN


### Let's resolve the Payment Method part first

In [14]:
res_df.groupby('Payment Method').agg({
    'Order ID':'count'
})

,Order ID
Payment Method,
Cash,5499
Credit Card,5504
Digital Wallet,5449


In [15]:
# Since we should not have any issues with cash payment, we can assume the issues (where data is not recorded properly) should be either card or digital payments
# so, we replace n/a values half by digital wallet and half by credit card

# at first, we find the index where n/a values are present
nan_indices = res_df[res_df['Payment Method'].isna()].index

# let's shuffle them now
nan_indices = np.random.permutation(nan_indices)

# divide into two halves
half = len(nan_indices) // 2
first_half = nan_indices[:half]
second_half = nan_indices[half:]

# assign value to each half

res_df.loc[first_half,'Payment Method'] = 'Credit Card'
res_df.loc[second_half,'Payment Method'] = 'Digital Wallet'

In [16]:
res_df.isna().sum()

Order ID             0
Customer ID          0
Category             0
Item              1758
Price              876
Quantity           430
Order Total        430
Order Date           0
Payment Method       0
dtype: int64

### Let's work on fixing the item along with its price and quantity 


In [17]:
res_df[(res_df['Item'].isna()) & (~res_df['Price'].isna()) & (~res_df['Quantity'].isna())]

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
7,ORD_146656,CUST_077,Main Dishes,NaN,15.0,3.0,45.0,2023-02-15,Cash
8,ORD_428611,CUST_083,Desserts,NaN,6.0,2.0,12.0,2023-12-16,Cash
18,ORD_337802,CUST_034,Desserts,NaN,4.0,5.0,20.0,2022-11-13,Credit Card
35,ORD_193762,CUST_028,Main Dishes,NaN,18.0,5.0,90.0,2022-04-08,Digital Wallet
51,ORD_782727,CUST_066,Starters,NaN,4.0,4.0,16.0,2022-08-11,Digital Wallet
...,...,...,...,...,...,...,...,...,...
17445,ORD_579430,CUST_016,Drinks,NaN,1.0,3.0,3.0,2022-07-25,Digital Wallet
17492,ORD_360934,CUST_080,Drinks,NaN,1.0,3.0,3.0,2022-12-21,Digital Wallet
17505,ORD_605079,CUST_089,Main Dishes,NaN,15.0,4.0,60.0,2022-12-23,Cash
17515,ORD_773375,CUST_024,Drinks,NaN,2.5,5.0,12.5,2022-11-24,Digital Wallet


In [18]:
# lets try and fix item 'dessert' and price 6

res_df[(res_df['Category'] == "Desserts") & (res_df['Price'] == 6)].groupby('Item').agg({
    'Quantity':'sum'
})

,Quantity
Item,
Brownie,1356.0
Chocolate Cake,2348.0


In [19]:
# Since the popular choice is chocolate cake for price = 6, we will use this item

res_df.loc[(res_df['Category'] == "Desserts") & (res_df['Price'] == 6),'Item'] = res_df.loc[(res_df['Category'] == "Desserts") & (res_df['Price'] == 6),'Item'].fillna("Chocolate Cake")

In [20]:
res_df[(res_df['Item'].isna()) & (res_df['Category'] == "Desserts") & (res_df['Price'] == 6)]

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method


In [21]:
res_df.isna().sum()

Order ID             0
Customer ID          0
Category             0
Item              1688
Price              876
Quantity           430
Order Total        430
Order Date           0
Payment Method       0
dtype: int64

In [22]:
# Null values have now come to 1688; Let's try groupby again

res_df[(res_df['Category'] == "Desserts") & (res_df['Price'] == 6)].groupby('Item').agg({
    'Quantity':'sum'
})


,Quantity
Item,
Brownie,1356.0
Chocolate Cake,2561.0


In [23]:
# Check other items as well 

In [24]:
res_df[(res_df['Item'].isna()) & (~res_df['Category'].isna()) & (~res_df['Price'].isna())]

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
7,ORD_146656,CUST_077,Main Dishes,NaN,15.0,3.0,45.0,2023-02-15,Cash
18,ORD_337802,CUST_034,Desserts,NaN,4.0,5.0,20.0,2022-11-13,Credit Card
35,ORD_193762,CUST_028,Main Dishes,NaN,18.0,5.0,90.0,2022-04-08,Digital Wallet
51,ORD_782727,CUST_066,Starters,NaN,4.0,4.0,16.0,2022-08-11,Digital Wallet
141,ORD_868890,CUST_007,Main Dishes,NaN,12.0,1.0,12.0,2022-10-28,Digital Wallet
...,...,...,...,...,...,...,...,...,...
17445,ORD_579430,CUST_016,Drinks,NaN,1.0,3.0,3.0,2022-07-25,Digital Wallet
17492,ORD_360934,CUST_080,Drinks,NaN,1.0,3.0,3.0,2022-12-21,Digital Wallet
17505,ORD_605079,CUST_089,Main Dishes,NaN,15.0,4.0,60.0,2022-12-23,Cash
17515,ORD_773375,CUST_024,Drinks,NaN,2.5,5.0,12.5,2022-11-24,Digital Wallet


In [25]:
# Let's figure out the mode items (most repetitive ones)

mode_items = (res_df.groupby(['Category','Price'])['Item']
              .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None))


In [26]:
mode_items

Category     Price
Desserts     4.0             Fruit Salad
             5.0               Ice Cream
             6.0          Chocolate Cake
             7.0              Cheesecake
Drinks       1.0                   Water
             2.5               Coca Cola
             3.0            Orange Juice
Main Dishes  12.0          Pasta Alfredo
             14.0     Vegetarian Platter
             15.0        Grilled Chicken
             18.0                 Salmon
             20.0                  Steak
Side Dishes  3.0              Side Salad
             4.0         Mashed Potatoes
             5.0      Grilled Vegetables
Starters     4.0            French Fries
             5.0            Cheese Fries
             7.0              Beef Chili
             8.0            Chicken Melt
             10.0          Nachos Grande
Name: Item, dtype: object

In [27]:
# we will now fill the null values with the most repetitive items based on category and price

In [28]:
res_df['Item'] = res_df['Item'].fillna(
    res_df[['Category','Price']].apply(tuple,axis=1).map(mode_items)
)

In [29]:
res_df.isna().sum()

Order ID            0
Customer ID         0
Category            0
Item              876
Price             876
Quantity          430
Order Total       430
Order Date          0
Payment Method      0
dtype: int64

### Let's now work on other data

In [30]:
res_df.describe()

,Price,Quantity,Order Total,Order Date
count,16658.000000,17104.000000,17104.000000,17534
mean,6.586325,3.014149,19.914494,2022-12-28 15:19:09.355538176
min,1.000000,1.000000,1.000000,2022-01-01 00:00:00
25%,3.000000,2.000000,7.500000,2022-06-30 00:00:00
50%,5.000000,3.000000,15.000000,2022-12-25 00:00:00
75%,7.000000,4.000000,25.000000,2023-06-29 00:00:00
max,20.000000,5.000000,100.000000,2023-12-31 00:00:00
std,4.834652,1.414598,18.732549,NaN


In [31]:
# In this case, it would be sensible to use median

# Identify rows where all three are NaN
mask = res_df[['Price', 'Quantity', 'Order Total']].isna().all(axis=1)

# Fill only those rows
res_df.loc[mask, 'Price'] = res_df['Price'].median()
res_df.loc[mask, 'Quantity'] = res_df['Quantity'].median()

In [32]:
res_df.isna().sum()

Order ID            0
Customer ID         0
Category            0
Item              876
Price             446
Quantity            0
Order Total       430
Order Date          0
Payment Method      0
dtype: int64

### We will now fix prices of data having total and quantity

In [33]:


res_df[(res_df['Price'].isna()) & (~res_df['Order Total'].isna()) & (~res_df['Quantity'].isna())]

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
3,ORD_630508,CUST_075,Drinks,NaN,NaN,2.0,5.0,2022-08-09,Credit Card
83,ORD_167858,CUST_056,Side Dishes,NaN,NaN,5.0,25.0,2022-04-01,Cash
112,ORD_289733,CUST_057,Drinks,NaN,NaN,5.0,5.0,2022-01-05,Digital Wallet
122,ORD_363522,CUST_070,Starters,NaN,NaN,1.0,8.0,2023-05-07,Credit Card
146,ORD_499199,CUST_079,Desserts,NaN,NaN,5.0,25.0,2022-02-12,Digital Wallet
...,...,...,...,...,...,...,...,...,...
17422,ORD_638163,CUST_080,Desserts,NaN,NaN,5.0,30.0,2022-06-11,Digital Wallet
17432,ORD_290007,CUST_088,Main Dishes,NaN,NaN,4.0,60.0,2022-06-22,Cash
17499,ORD_118085,CUST_089,Drinks,NaN,NaN,4.0,12.0,2023-04-09,Digital Wallet
17509,ORD_239707,CUST_056,Starters,NaN,NaN,1.0,8.0,2023-07-27,Credit Card


In [34]:
mask1 = res_df['Price'].isna() & res_df['Order Total'].notna() & res_df['Quantity'].notna()

In [35]:
res_df.loc[mask1,'Price'] = res_df.loc[mask1,'Order Total'] / res_df.loc[mask1,'Quantity']


In [36]:
res_df.isna().sum()

Order ID            0
Customer ID         0
Category            0
Item              876
Price               0
Quantity            0
Order Total       430
Order Date          0
Payment Method      0
dtype: int64

In [37]:
res_df[res_df['Order Total'].isna() & res_df['Price'].notna() & res_df['Quantity'].notna()]

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
75,ORD_367023,CUST_055,Side Dishes,NaN,5.0,3.0,NaN,2022-05-29,Credit Card
118,ORD_627458,CUST_058,Side Dishes,NaN,5.0,3.0,NaN,2023-08-24,Digital Wallet
131,ORD_467157,CUST_099,Drinks,NaN,5.0,3.0,NaN,2023-11-30,Digital Wallet
240,ORD_467033,CUST_032,Side Dishes,NaN,5.0,3.0,NaN,2023-03-04,Digital Wallet
253,ORD_722632,CUST_027,Desserts,NaN,5.0,3.0,NaN,2022-12-11,Credit Card
...,...,...,...,...,...,...,...,...,...
17410,ORD_882909,CUST_023,Drinks,NaN,5.0,3.0,NaN,2023-07-11,Credit Card
17451,ORD_529422,CUST_068,Drinks,NaN,5.0,3.0,NaN,2022-07-17,Credit Card
17473,ORD_342327,CUST_096,Desserts,NaN,5.0,3.0,NaN,2022-12-29,Cash
17497,ORD_205432,CUST_090,Drinks,NaN,5.0,3.0,NaN,2022-03-03,Cash


### Now the total amount

In [38]:


mask2 = res_df['Order Total'].isna() & res_df['Price'].notna() & res_df['Quantity'].notna()

res_df.loc[mask2,'Order Total'] = res_df.loc[mask2,'Price'] * res_df.loc[mask2,'Quantity']

In [41]:
res_df.isna().sum()

Order ID            0
Customer ID         0
Category            0
Item              876
Price               0
Quantity            0
Order Total         0
Order Date          0
Payment Method      0
dtype: int64

In [42]:
res_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17534 entries, 0 to 17533
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Order ID        17534 non-null  object        
 1   Customer ID     17534 non-null  object        
 2   Category        17534 non-null  object        
 3   Item            16658 non-null  object        
 4   Price           17534 non-null  float64       
 5   Quantity        17534 non-null  float64       
 6   Order Total     17534 non-null  float64       
 7   Order Date      17534 non-null  datetime64[ns]
 8   Payment Method  17534 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 1.2+ MB


In [43]:
res_df.isna().sum()

Order ID            0
Customer ID         0
Category            0
Item              876
Price               0
Quantity            0
Order Total         0
Order Date          0
Payment Method      0
dtype: int64

In [44]:
res_df['Item'] = res_df['Item'].fillna("Out of Menu")

In [45]:
res_df.isna().sum()

Order ID          0
Customer ID       0
Category          0
Item              0
Price             0
Quantity          0
Order Total       0
Order Date        0
Payment Method    0
dtype: int64

In [46]:
res_df.to_csv("C:/Users/Bibek/OneDrive/Desktop/DA Training/Restaurant Sales Data Cleaning/restaurant_clean.csv")